# triggering render jobs

### Setup

Start by running $WORKSPACE/tools/fission/reset-fission-proxies before launching the notebook server.

In [74]:
# GCP project to use
PROJECT="kubeflow-rl"

# Bucket to use
BUCKET=PROJECT+"-kf"

# K8s cluster to use
CLUSTER="kubeflow-5fc52116"
ZONE="us-east1-d"
NAMESPACE="rl"

ROOT_DIR = "/Users/cb/code/dancing-water/kubeflow-rl"

#### (Re)deploy function and trigger

In [9]:
%%bash
sh deploy.sh

function 'job-trigger-render' deleted
trigger 'ebd9b767-1fc0-4428-b5cd-d8343019a6b5' deleted
function 'job-trigger-render' created
trigger '46ef36d4-12b0-412e-9456-9175b6619a42' created


#### Update and trigger function

In [64]:
%%bash
sh dev.sh

function 'job-trigger-render' updated
{'api_version': 'batch/v1',
 'kind': 'Job',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2018, 1, 22, 17, 40, 9, tzinfo=tzlocal()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'initializers': None,
              'labels': {'controller-uid': '4a8c23c4-ff9b-11e7-b604-42010af00218',
                         'job-name': 'pybullet-kuka-ff-0118-2346-bac2-render-cf11f2dc'},
              'name': 'pybullet-kuka-ff-0118-2346-bac2-render-cf11f2dc',
              'namespace': 'rl',
              'owner_references': None,
              'resource_version': '1780363',
              'self_link': '/apis/batch/v1/namespaces/rl/jobs/pybullet-kuka-ff-0118-2346-bac2-render-cf11f2dc',
              'uid': '4a8c23c4-ff9b-11e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6203  100  6113  100    90  19802    291 --:--:-- --:--:-- --:--:-- 19847


In [72]:
%%bash
kubectl get pods -n rl

NAME                                                    READY     STATUS    RESTARTS   AGE
pybullet-kuka-ff-0118-2346-bac2-render-cf11f2dc-vg8rc   1/1       Running   0          28s


In [95]:
%%bash
kubectl logs pybullet-kuka-ff-0118-2346-bac2-render-cf11f2dc-vg8rc -n rl

INFO:tensorflow:Resume run and write summaries and checkpoints to gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2.
INFO:tensorflow:Resume run and write summaries and checkpoints to gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2.
{'algorithm': <class 'agents.ppo.algorithm.PPOAlgorithm'>,
 'discount': 0.995,
 'env': 'KukaBulletEnv-v0',
 'eval_episodes': 25,
 'init_logstd': -1,
 'init_mean_factor': 0.1,
 'kl_cutoff_coef': 1000,
 'kl_cutoff_factor': 2,
 'kl_init_penalty': 1,
 'kl_target': 0.01,
 'learning_rate': 0.0001,
 'logdir': 'gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2',
 'max_length': 1000,
 'network': <function feed_forward_gaussian at 0x7f0afc908b18>,
 'num_agents': 30,
 'optimizer': <class 'tensorflow.python.training.adam.AdamOptimizer'>,
 'optimizer_pre_initialize': True,
 'policy_layers': (200, 100),
 'steps': 40000000.0,
 'update_epochs': 25,
 'update_every': 30,
 'use_gpu': False,
 'value_layers': (200, 100),
 'weight_summaries': {'all': '.*',


In [96]:
%%bash
gsutil ls gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render

gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.episode_batch.0.46.stats.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.manifest.0.46.manifest.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.video.0.46.video000000.meta.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.video.0.46.video000000.mp4
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.video.0.46.video000001.meta.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.video.0.46.video000001.mp4
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.video.0.46.video000002.meta.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.video.0.46.video000002.mp4
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.video.0.46.video000003.meta.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render

Get the cluster-internal IP of the Fission router pod

In [191]:
!kubectl -n fission get pod -o wide|grep router | sed -E 's/[[:space:]]+/ /g' | cut -f6 -d' '

10.28.0.98


Create a job on kubeflow showing display of job trigger request every N seconds (not submitted).

In [186]:
%%bash

get_project_id() {
  project=$(gcloud config get-value project 2> /dev/null)
  if [[ -z "$project" ]]; then
      >&2 echo "Couldn't load a gcloud project ID!"
  fi
  echo "$project"
}

cd ../../../rl-app


HPARAM_ID=kuka

JOB_SALT=`date | shasum -a 256 | cut -c1-8`
JOB_NAME=`echo ${HPARAM_ID}-${JOB_SALT} | tr '_' '-'`
LOG_DIR=gs://$(get_project_id)-kf/jobs/${JOB_NAME}

# 1.4.1, agents propper, MTS, periodic render messages
IMAGE=gcr.io/kubeflow-rl/agents-ppo:cpu-6ea29f81

ks param set agents-ppo env "KukaBulletEnv-v0"

ks param set agents-ppo run_mode train
ks param set agents-ppo gcp_project kubeflow-rl
ks param set agents-ppo num_cpu 1
ks param set agents-ppo num_agents 1
ks param set agents-ppo sync_replicas False
ks param set agents-ppo steps 1000

# Trigger an async render job every 10 seconds (for demo)
ks param set agents-ppo render_secs 10
ks param set agents-ppo fission_router_ip "10.28.0.98"

ks param set agents-ppo algorithm "agents.ppo.PPOAlgorithm"
ks param set agents-ppo network "agents.scripts.networks.feed_forward_gaussian"

ks param set agents-ppo job_tag ${JOB_SALT}
ks param set agents-ppo logdir ${LOG_DIR}
ks param set agents-ppo name ${JOB_NAME}
ks param set agents-ppo image ${IMAGE}

ks apply gke -c agents-ppo

Parameter 'env' successfully set to '"KukaBulletEnv-v0"' for component 'agents-ppo'
Parameter 'run_mode' successfully set to '"train"' for component 'agents-ppo'
Parameter 'gcp_project' successfully set to '"kubeflow-rl"' for component 'agents-ppo'
Parameter 'num_cpu' successfully set to '1' for component 'agents-ppo'
Parameter 'num_agents' successfully set to '1' for component 'agents-ppo'
Parameter 'sync_replicas' successfully set to '"False"' for component 'agents-ppo'
Parameter 'steps' successfully set to '1000' for component 'agents-ppo'
Parameter 'render_secs' successfully set to '10' for component 'agents-ppo'
Parameter 'fission_router_ip' successfully set to '"10.28.0.98"' for component 'agents-ppo'
Parameter 'algorithm' successfully set to '"agents.ppo.PPOAlgorithm"' for component 'agents-ppo'
Parameter 'network' successfully set to '"agents.scripts.networks.feed_forward_gaussian"' for component 'agents-ppo'
Parameter 'job_tag' successfully set to '"a9bd0dee"' for component 'a

In [192]:
%%bash
kubectl get pods -n rl

NAME                                READY     STATUS    RESTARTS   AGE
kuka-a9bd0dee-master-rf93-0-pxp97   1/1       Running   0          51s
ubuntu                              1/1       Running   0          1h


Note that in the following elasped time increments by render_secs

In [195]:
%%bash
kubectl logs kuka-a9bd0dee-master-rf93-0-pxp97 -n rl

INFO:tensorflow:Start a new run and write summaries and checkpoints to gs://kubeflow-rl-kf/jobs/kuka-a9bd0dee.
2018-01-23 21:09:47.178438: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
E0123 21:09:47.248892495       1 ev_epoll1_linux.c:1051]     grpc epoll fd: 3
2018-01-23 21:09:47.258648: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:215] Initialize GrpcChannelCache for job master -> {0 -> localhost:2222}
2018-01-23 21:09:47.262287: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:324] Started server with target: grpc://localhost:2222
{'algorithm': <class 'agents.ppo.algorithm.PPOAlgorithm'>,
 'debug': True,
 'discount': 0.995,
 'dump_dependency_versions': False,
 'env': 'KukaBulletEnv-v0',
 'env_processes': True,
 'eval_episodes': 25,
 'fission_router_ip': '10.28.0.98',
 'hparam_set_id': 'pybullet_kuka_ff',
 'init_logstd': -1,
 'init_mean_facto

Actually trigger jobs?

In [18]:

%%bash

get_project_id() {
  project=$(gcloud config get-value project 2> /dev/null)
  if [[ -z "$project" ]]; then
      >&2 echo "Couldn't load a gcloud project ID!"
  fi
  echo "$project"
}

cd ../../../rl-app


HPARAM_ID=kuka

JOB_SALT=`date | shasum -a 256 | cut -c1-8`
JOB_NAME=`echo ${HPARAM_ID}-${JOB_SALT} | tr '_' '-'`
LOG_DIR=gs://$(get_project_id)-kf/jobs/${JOB_NAME}

# 1.4.1, agents propper, MTS, periodic render messages
IMAGE=gcr.io/kubeflow-rl/agents-ppo:cpu-df1d364a

ks param set agents-ppo env "KukaBulletEnv-v0"

ks param set agents-ppo run_mode train
ks param set agents-ppo gcp_project kubeflow-rl
ks param set agents-ppo num_cpu 1
ks param set agents-ppo num_agents 1
ks param set agents-ppo sync_replicas False
ks param set agents-ppo steps 1000

# Trigger an async render job every 10 seconds (for demo)
ks param set agents-ppo render_secs 10

ks param set agents-ppo fission_router_ip "10.28.0.98"

ks param set agents-ppo algorithm "agents.ppo.PPOAlgorithm"
ks param set agents-ppo network "agents.scripts.networks.feed_forward_gaussian"

ks param set agents-ppo job_tag ${JOB_SALT}
ks param set agents-ppo logdir ${LOG_DIR}
ks param set agents-ppo name ${JOB_NAME}
ks param set agents-ppo image ${IMAGE}

ks apply gke -c agents-ppo

Parameter 'env' successfully set to '"KukaBulletEnv-v0"' for component 'agents-ppo'
Parameter 'run_mode' successfully set to '"train"' for component 'agents-ppo'
Parameter 'gcp_project' successfully set to '"kubeflow-rl"' for component 'agents-ppo'
Parameter 'num_cpu' successfully set to '1' for component 'agents-ppo'
Parameter 'num_agents' successfully set to '1' for component 'agents-ppo'
Parameter 'sync_replicas' successfully set to '"False"' for component 'agents-ppo'
Parameter 'steps' successfully set to '1000' for component 'agents-ppo'
Parameter 'render_secs' successfully set to '10' for component 'agents-ppo'
Parameter 'fission_router_ip' successfully set to '"10.28.0.98"' for component 'agents-ppo'
Parameter 'algorithm' successfully set to '"agents.ppo.PPOAlgorithm"' for component 'agents-ppo'
Parameter 'network' successfully set to '"agents.scripts.networks.feed_forward_gaussian"' for component 'agents-ppo'
Parameter 'job_tag' successfully set to '"355e5095"' for component 'a

In [21]:
%%bash
kubectl get pods -n rl

NAME      READY     STATUS    RESTARTS   AGE
ubuntu    1/1       Running   0          3h


In [22]:
%%bash
kubectl logs kuka-355e5095-master-xozz-0-mnxn5 -n rl

INFO:tensorflow:Start a new run and write summaries and checkpoints to gs://kubeflow-rl-kf/jobs/kuka-355e5095.
2018-01-23 23:07:07.389503: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
E0123 23:07:07.412911008       1 ev_epoll1_linux.c:1051]     grpc epoll fd: 3
2018-01-23 23:07:07.422978: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:215] Initialize GrpcChannelCache for job master -> {0 -> localhost:2222}
2018-01-23 23:07:07.426076: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:324] Started server with target: grpc://localhost:2222
{'algorithm': <class 'agents.ppo.algorithm.PPOAlgorithm'>,
 'debug': True,
 'discount': 0.995,
 'dump_dependency_versions': False,
 'env': 'KukaBulletEnv-v0',
 'env_processes': True,
 'eval_episodes': 25,
 'hparam_set_id': 'pybullet_kuka_ff',
 'init_logstd': -1,
 'init_mean_factor': 0.1,
 'kl_cutoff_coef': 1000,
 '

In [23]:
%%bash
kubectl get jobs -n rl

NAME                          DESIRED   SUCCESSFUL   AGE
kuka-355e5095-master-xozz-0   1         1            2m


In [94]:
%%bash
kubectl delete tfjobs --all -n rl 
kubectl delete jobs --all -n rl 

tfjob "kuka-0123-1549-272b" deleted
job "kuka-46613b75-render-2" deleted


Now, lets have the training job actually submit render jobs each time that hook is triggered.

In [77]:
%%bash
kubectl get jobs -n rl

No resources found.


In [82]:
import datetime
import uuid
import os

os.chdir(os.path.join(ROOT_DIR, "rl-app"))

HPARAM_SET="kuka"

now=datetime.datetime.now()

JOB_SALT=now.strftime("%m%d-%H%M") + "-" + uuid.uuid4().hex[0:4]
JOB_NAME=HPARAM_SET + "-" + JOB_SALT
LOG_DIR="gs://{0}/jobs/{1}".format(BUCKET, JOB_NAME)

IMAGE="gcr.io/kubeflow-rl/agents-ppo:cpu-35b72200"

print(JOB_SALT)
print(JOB_NAME)
print(LOG_DIR)
print(IMAGE)

!ks param set agents-ppo env "KukaBulletEnv-v0"

!ks param set agents-ppo run_mode train
!ks param set agents-ppo gcp_project kubeflow-rl
!ks param set agents-ppo num_cpu 1
!ks param set agents-ppo num_agents 1
!ks param set agents-ppo sync_replicas False
!ks param set agents-ppo steps 1000

# Trigger an async render job every 10 seconds (for demo)
!ks param set agents-ppo render_secs 10

!ks param set agents-ppo fission_router_ip "10.28.0.98"

!ks param set agents-ppo algorithm "agents.ppo.PPOAlgorithm"
!ks param set agents-ppo network "agents.scripts.networks.feed_forward_gaussian"

!ks param set agents-ppo job_tag {JOB_SALT}
!ks param set agents-ppo logdir {LOG_DIR}
!ks param set agents-ppo name {JOB_NAME}
!ks param set agents-ppo image {IMAGE}

!ks apply gke -c agents-ppo

0123-1541-4a69
kuka-0123-1541-4a69
gs://kubeflow-rl-kf/jobs/kuka-0123-1541-4a69
gcr.io/kubeflow-rl/agents-ppo:cpu-35b72200
INFO  Parameter 'env' successfully set to '"KukaBulletEnv-v0"' for component 'agents-ppo'
INFO  Parameter 'run_mode' successfully set to '"train"' for component 'agents-ppo'
INFO  Parameter 'gcp_project' successfully set to '"kubeflow-rl"' for component 'agents-ppo'
INFO  Parameter 'num_cpu' successfully set to '1' for component 'agents-ppo'
INFO  Parameter 'num_agents' successfully set to '1' for component 'agents-ppo'
INFO  Parameter 'sync_replicas' successfully set to '"False"' for component 'agents-ppo'
INFO  Parameter 'steps' successfully set to '1000' for component 'agents-ppo'
INFO  Parameter 'render_secs' successfully set to '10' for component 'agents-ppo'
INFO  Parameter 'fission_router_ip' successfully set to '"10.28.0.98"' for component 'agents-ppo'
INFO  Parameter 'algorithm' successfully set to '"agents.ppo.PPOAlgorithm"' for component 'agents-ppo'
INF

In [83]:
%%bash
kubectl get pods -n rl

NAME                                      READY     STATUS    RESTARTS   AGE
kuka-0123-1541-4a69-master-6nff-0-krr5h   1/1       Running   0          14s
ubuntu                                    1/1       Running   0          4h


In [89]:
!kubectl logs kuka-0123-1541-4a69-master-6nff-0-krr5h -n rl

INFO:tensorflow:Start a new run and write summaries and checkpoints to gs://kubeflow-rl-kf/jobs/kuka-0123-1541-4a69.
2018-01-23 23:41:58.461261: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
E0123 23:41:58.549969808       1 ev_epoll1_linux.c:1051]     grpc epoll fd: 3
2018-01-23 23:41:58.559454: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:215] Initialize GrpcChannelCache for job master -> {0 -> localhost:2222}
2018-01-23 23:41:58.562608: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:324] Started server with target: grpc://localhost:2222
{'algorithm': <class 'agents.ppo.algorithm.PPOAlgorithm'>,
 'debug': True,
 'discount': 0.995,
 'dump_dependency_versions': False,
 'env': 'KukaBulletEnv-v0',
 'env_processes': True,
 'eval_episodes': 25,
 'fission_router_ip': '10.28.0.98',
 'hparam_set_id': 'pybullet_kuka_ff',
 'init_logstd': -1,
 'init_mean

In [90]:
!kubectl get jobs -n rl

NAME                                DESIRED   SUCCESSFUL   AGE
kuka-0123-1541-4a69-master-6nff-0   1         0            52s
kuka-0123-1541-4a69-render-1        1         0            12s
kuka-0123-1541-4a69-render-2        1         0            3s


Cool! 🎉🎉🎉

We have the option here of triggering renders based on time, checkpoint number, or integrating with the CheckpointSaverHook to trigger renders each time a checkpoint is written which will allow us to only render when there are new parameters available to be used in a render. (probably the right way to go!)

Also it's debatable whether you would want to trigger renders over HTTP instead of directly from the training job but after experimenting with this I'm leaning strongly towards the latter.

#### Long training run with reasonable render periodicity

Lets try a long training run where renders are produced every 10 minutes (the same periodicity of our checkpoint writes).

In [96]:
import datetime
import uuid
import os

os.chdir(os.path.join(ROOT_DIR, "rl-app"))

HPARAM_SET="kuka"

now=datetime.datetime.now()

JOB_SALT=now.strftime("%m%d-%H%M") + "-" + uuid.uuid4().hex[0:4]
JOB_NAME=HPARAM_SET + "-" + JOB_SALT
LOG_DIR="gs://{0}/jobs/{1}".format(BUCKET, JOB_NAME)

IMAGE="gcr.io/kubeflow-rl/agents-ppo:cpu-256e4ec7"

print(JOB_SALT)
print(JOB_NAME)
print(LOG_DIR)
print(IMAGE)

!ks param set agents-ppo env "KukaBulletEnv-v0"

!ks param set agents-ppo run_mode train
!ks param set agents-ppo gcp_project kubeflow-rl
!ks param set agents-ppo num_cpu 31
!ks param set agents-ppo num_agents 30
!ks param set agents-ppo sync_replicas False
!ks param set agents-ppo steps 4e7

# Trigger an async render job every 10 seconds (for demo)
!ks param set agents-ppo render_secs 610

!ks param set agents-ppo fission_router_ip "10.28.0.98"

!ks param set agents-ppo algorithm "agents.ppo.PPOAlgorithm"
!ks param set agents-ppo network "agents.scripts.networks.feed_forward_gaussian"

!ks param set agents-ppo job_tag {JOB_SALT}
!ks param set agents-ppo logdir {LOG_DIR}
!ks param set agents-ppo name {JOB_NAME}
!ks param set agents-ppo image {IMAGE}

!ks apply gke -c agents-ppo

0123-1611-753a
kuka-0123-1611-753a
gs://kubeflow-rl-kf/jobs/kuka-0123-1611-753a
gcr.io/kubeflow-rl/agents-ppo:cpu-256e4ec7
INFO  Parameter 'env' successfully set to '"KukaBulletEnv-v0"' for component 'agents-ppo'
INFO  Parameter 'run_mode' successfully set to '"train"' for component 'agents-ppo'
INFO  Parameter 'gcp_project' successfully set to '"kubeflow-rl"' for component 'agents-ppo'
INFO  Parameter 'num_cpu' successfully set to '31' for component 'agents-ppo'
INFO  Parameter 'num_agents' successfully set to '30' for component 'agents-ppo'
INFO  Parameter 'sync_replicas' successfully set to '"False"' for component 'agents-ppo'
INFO  Parameter 'steps' successfully set to '4e7' for component 'agents-ppo'
INFO  Parameter 'render_secs' successfully set to '610' for component 'agents-ppo'
INFO  Parameter 'fission_router_ip' successfully set to '"10.28.0.98"' for component 'agents-ppo'
INFO  Parameter 'algorithm' successfully set to '"agents.ppo.PPOAlgorithm"' for component 'agents-ppo'
I

In [97]:
!kubectl get pods -n rl

NAME                                      READY     STATUS    RESTARTS   AGE
kuka-0123-1611-753a-master-tgry-0-vsz7c   1/1       Running   0          10s
ubuntu                                    1/1       Running   0          4h


In [103]:
!kubectl logs -n rl kuka-0123-1611-753a-master-tgry-0-vsz7c

INFO:tensorflow:Start a new run and write summaries and checkpoints to gs://kubeflow-rl-kf/jobs/kuka-0123-1611-753a.
2018-01-24 00:11:54.754763: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
E0124 00:11:54.756339511       1 ev_epoll1_linux.c:1051]     grpc epoll fd: 3
2018-01-24 00:11:54.767196: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:215] Initialize GrpcChannelCache for job master -> {0 -> localhost:2222}
2018-01-24 00:11:54.771314: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:324] Started server with target: grpc://localhost:2222
{'algorithm': <class 'agents.ppo.algorithm.PPOAlgorithm'>,
 'debug': True,
 'discount': 0.995,
 'dump_dependency_versions': False,
 'env': 'KukaBulletEnv-v0',
 'env_processes': True,
 'eval_episodes': 25,
 'fission_router_ip': '10.28.0.98',
 'hparam_set_id': 'pybullet_kuka_ff',
 'init_logstd': -1,
 'init_mean

In [107]:
!kubectl get jobs -n rl

NAME                                DESIRED   SUCCESSFUL   AGE
kuka-0123-1611-753a-master-tgry-0   1         0            2m


In [108]:
!gsutil ls {LOG_DIR}/render

CommandException: One or more URLs matched no objects.


### Status:
- Currently finishes without error but renders do not arrive on cloud storage.
- Should be rendering to /tmp then uploading render when complete